# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Liu  ->  J. Liu  |  ['J. Liu']


A. Müller  ->  A. Müller  |  ['A. Müller']
T. Müller  ->  T. Müller  |  ['T. Müller']
M. Schirmer  ->  M. Schirmer  |  ['M. Schirmer']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Paterson  ->  K. Paterson  |  ['K. Paterson']
M. Schirmer  ->  M. Schirmer  |  ['M. Schirmer']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Paterson  ->  K. Paterson  |  ['K. Paterson']
M. Schirmer  ->  M. Schirmer  |  ['M. Schirmer']


K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']


K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']


K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']


K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']


K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
M. Schirmer  ->  M. Schirmer  |  ['M. Schirmer']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Paterson  ->  K. Paterson  |  ['K. Paterson']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']


K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
C. Gapp  ->  C. Gapp  |  ['C. Gapp']
K. Paschek  ->  K. Paschek  |  ['K. Paschek']
T. K. Henning  ->  T. K. Henning  |  ['T. K. Henning']
Arxiv has 98 new papers today
          30 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/30 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2503.14726


extracting tarball to tmp_2503.14726... done.
Retrieving document from  https://arxiv.org/e-print/2503.15067


extracting tarball to tmp_2503.15067...

 done.
Retrieving document from  https://arxiv.org/e-print/2503.15081


...

/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure W178b_jwstonly_free_corner.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=1.0\linewidth]{W178b_jwstonly_free_corner.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure W178b_hstjwst_free_corner_v3.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=1.0\linewidth]{W178b_hstjwst_free_corner_v3.pdf}
  warnings.warn(LatexWarning(f"Could not fi

Issues with the citations
list index out of range
Retrieving document from  https://arxiv.org/e-print/2503.15479
extracting tarball to tmp_2503.15479...

 done.


K. Paschek  ->  K. Paschek  |  ['K. Paschek']
T. K. Henning  ->  T. K. Henning  |  ['T. K. Henning']


Found 199 bibliographic references in tmp_2503.15479/main.bbl.
Error retrieving bib data for Miyazaki2022: 'author'
Error retrieving bib data for Pearce2022: 'author'
Error retrieving bib data for Pearce2022: 'author'


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.15302-b31b1b.svg)](https://arxiv.org/abs/2503.15302) | **Euclid Quick Data Release (Q1) - Data release overview**  |
|| E. Collaboration, et al. -- incl., <mark>M. Schirmer</mark>, <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-03-20*|
|*Comments*| *27 pages, 12 figures, data release at this https URL paper submitted to the special A&A issue*|
|**Abstract**|            The first Euclid Quick Data Release, Q1, comprises 63.1 sq deg of the Euclid Deep Fields (EDFs) to nominal wide-survey depth. It encompasses visible and near-infrared space-based imaging and spectroscopic data, ground-based photometry in the u, g, r, i and z bands, as well as corresponding masks. Overall, Q1 contains about 30 million objects in three areas near the ecliptic poles around the EDF-North and EDF-South, as well as the EDF-Fornax field in the constellation of the same name. The purpose of this data release -- and its associated technical papers -- is twofold. First, it is meant to inform the community of the enormous potential of the Euclid survey data, to describe what is contained in these data, and to help prepare expectations for the forthcoming first major data release DR1. Second, it enables a wide range of initial scientific projects with wide-survey Euclid data, ranging from the early Universe to the Solar System. The Q1 data were processed with early versions of the processing pipelines, which already demonstrate good performance, with numerous improvements in implementation compared to pre-launch development. In this paper, we describe the sky areas released in Q1, the observations, a top-level view of the data processing of Euclid and associated external data, the Q1 photometric masks, and how to access the data. We also give an overview of initial scientific results obtained using the Q1 data set by Euclid Consortium scientists, and conclude with important caveats when using the data. As a complementary product, Q1 also contains observations of a star-forming area in Lynd's Dark Nebula 1641 in the Orion~A Cloud, observed for technical purposes during Euclid's performance-verification phase. This is a unique target, of a type not commonly found in Euclid's nominal sky survey.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.15303-b31b1b.svg)](https://arxiv.org/abs/2503.15303) | **Euclid Quick Data Release (Q1): VIS processing and data products**  |
|| E. Collaboration, et al. -- incl., <mark>K. Paterson</mark>, <mark>M. Schirmer</mark> |
|*Appeared on*| *2025-03-20*|
|*Comments*| *Paper submitted as part of the A&A Special Issue `Euclid Quick Data Release (Q1)', 21 pages, 21 figures*|
|**Abstract**|            This paper describes the VIS Processing Function (VIS PF) of the Euclid ground segment pipeline, which processes and calibrates raw data from the VIS camera. We present the algorithms used in each processing element, along with a description of the on-orbit performance of VIS PF, based on Performance Verification (PV) and Q1 data. We demonstrate that the principal performance metrics (image quality, astrometric accuracy, photometric calibration) are within pre-launch specifications. The image-to-image photometric scatter is less than $0.8\%$, and absolute astrometric accuracy compared to Gaia is $5$ mas Image quality is stable over all Q1 images with a full width at half maximum (FWHM) of $0.\!^{\prime\prime}16$. The stacked images (combining four nominal and two short exposures) reach $I_\mathrm{E} = 25.6$ ($10\sigma$, measured as the variance of $1.\!^{\prime\prime}3$ diameter apertures). We also describe quality control metrics provided with each image, and an appendix provides a detailed description of the provided data products. The excellent quality of these images demonstrates the immense potential of Euclid VIS data for weak lensing. VIS data, covering most of the extragalactic sky, will provide a lasting high-resolution atlas of the Universe.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.15304-b31b1b.svg)](https://arxiv.org/abs/2503.15304) | **Euclid Quick Data Release (Q1). NIR processing and data products**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>, <mark>K. Paterson</mark>, <mark>M. Schirmer</mark> |
|*Appeared on*| *2025-03-20*|
|*Comments*| *24 pages, 18 figures*|
|**Abstract**|            This paper describes the near-infrared processing function (NIR PF) that processes near-infrared images from the Near-Infrared Spectrometer and Photometer (NISP) instrument onboard the Euclid satellite. NIR PF consists of three main components: (i) a common pre-processing stage for both photometric (NIR) and spectroscopic (SIR) data to remove instrumental effects; (ii) astrometric and photometric calibration of NIR data, along with catalogue extraction; and (iii) resampling and stacking. The necessary calibration products are generated using dedicated pipelines that process observations from both the early performance verification (PV) phase in 2023 and the nominal survey operations. After outlining the pipeline's structure and algorithms, we demonstrate its application to Euclid Q1 images. For Q1, we achieve an astrometric accuracy of 9-15 mas, a relative photometric accuracy of 5 mmag, and an absolute flux calibration limited by the 1% uncertainty of the Hubble Space Telescope (HST) CALSPEC database. We characterise the point-spread function (PSF) that we find very stable across the focal plane, and we discuss current limitations of NIR PF that will be improved upon for future data releases.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.15309-b31b1b.svg)](https://arxiv.org/abs/2503.15309) | **Euclid Quick Data Release (Q1). Exploring galaxy morphology across cosmic time through Sersic fits**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-03-20*|
|*Comments*| *Paper submitted as part of the A&A Special Issue `Euclid Quick Data Release (Q1)', 23 pages, 16 figures*|
|**Abstract**|            We present the results of the single-component Sérsic profile fitting for the magnitude-limited sample of \IE$<23$ galaxies within the 63.1 deg$^2$ area of the Euclid Quick Data Release (Q1). The associated morphological catalogue includes two sets of structural parameters fitted using \texttt{SourceXtractor++}: one for VIS \IE images and one for a combination of three NISP images in \YE, \JE and \HE bands. We compare the resulting Sérsic parameters to other morphological measurements provided in the Q1 data release, and to the equivalent parameters based on higher-resolution \HST imaging. These comparisons confirm the consistency and the reliability of the fits to Q1 data. Our analysis of colour gradients shows that NISP profiles have systematically smaller effective radii ($R_{\rm e}$) and larger Sérsic indices ($n$) than in VIS. In addition, we highlight trends in NISP-to-VIS parameter ratios with both magnitude and $n_{\rm VIS}$. From the 2D bimodality of the $(u-r)$ colour-$\log(n)$ plane, we define a $(u-r)_{\rm lim}(n)$ that separates early- and late-type galaxies (ETGs and LTGs). We use the two subpopulations to examine the variations of $n$ across well-known scaling relations at $z<1$. ETGs display a steeper size--stellar mass relation than LTGs, indicating a difference in the main drivers of their mass assembly. Similarly, LTGs and ETGs occupy different parts of the stellar mass--star-formation rate plane, with ETGs at higher masses than LTGs, and further down below the Main Sequence of star-forming galaxies. This clear separation highlights the link known between the shutdown of star formation and morphological transformations in the Euclid imaging data set. In conclusion, our analysis demonstrates both the robustness of the Sérsic fits available in the Q1 morphological catalogue and the wealth of information they provide for studies of galaxy evolution with Euclid.         |

...

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.15081-b31b1b.svg)](https://arxiv.org/abs/2503.15081) | **Orbits of very distant asteroid satellites**  |
|| K. Minker, et al. -- incl., <mark>T. Müller</mark> |
|*Appeared on*| *2025-03-20*|
|*Comments*| *22 pages, 8 figures, 16 tables, accepted for publication at A&A*|
|**Abstract**|            The very wide binary asteroid (VWBA) population is a small subset of the population of known binary and multiple asteroids made of systems with very widely orbiting satellites and long orbital periods, on the order of tens to hundreds of days. The origin of these systems is debatable, and most members of this population are poorly characterized. We have compiled all available high-angular resolution imaging archival data of VWBA systems from large ground- and space-based telescopes. We measure the astrometric positions of the satellite relative to the primary and analyze the dynamics of the satellites using the Genoid genetic algorithm. Additionally, we use a NEATM thermal model to estimate the diameters of two systems, and we model the orbit of Litva's inner satellite using photometric lightcurve observations. We determine the effective diameters of binary systems Christophedumas and Alconrad to be 4.7 + 0.4km and 5.2 + 0.3km respectively. We determine new orbital solutions for five systems, Huenna, Litva, (3548) Eurybates, Pauling, and Alconrad. We find a significantly eccentric best-fit orbital solution for the outer satellite of Litva, a moderately eccentric solution for Alconrad, and a nearly circular solution for Pauling. We also confirm previously reported orbital solutions for (379) Huenna and Eurybates. It is unlikely that BYORP expansion could be solely responsible for the formation of VWBAs. It is possible that the satellites of these systems were formed through YORP spin-up and then later scattered onto very wide orbits. Additionally, we find that some members of the population are unlikely to have formed satellites through YORP spin-up, and a collisional formation history is favored. In particular, this applies to VWBAs within large dynamical families, or large VWBA systems such as Huenna and NASA's Lucy mission target Eurybates.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.14726-b31b1b.svg)](https://arxiv.org/abs/2503.14726) | **Characterizing Magnetic Properties of Young Protostars in Orion**  |
|| B. Huang, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2025-03-20*|
|*Comments*| *12 pages, 2 figures, 4 tables (accepted for publication in The Astrophysical Journal)*|
|**Abstract**|            The {\em B}-field Orion Protostellar Survey (BOPS) recently obtained polarimetric observations at 870 ${\rm \mu m}$ towards 61 protostars in the Orion molecular clouds with $\sim 1^{\prime\prime}$ spatial resolution using the Atacama Large Millimeter/submillimeter Array. From the BOPS sample, we selected the 26 protostars with extended polarized emission within a radius of $\sim 6^{\prime\prime}$ (2400~au) around the protostar. This allows to have sufficient statistical polarization data to infer the magnetic field strength. The magnetic field strength is derived using the Davis-Chandrasekhar-Fermi method. The underlying magnetic field strengths are approximately 2.0~mG for protostars with a standard hourglass magnetic field morphology, which is higher than the values derived for protostars with rotated hourglass, spiral, and complex magnetic field configurations ($\lesssim1.0$~mG). This suggests that the magnetic field plays a more significant role in envelopes exhibiting a standard hourglass field morphology, and a value of $\gtrsim2.0$ mG would be required to maintain such a structure at these scales. Furthermore, most protostars in the sample are slightly supercritical, with mass-to-flux ratios $\lesssim3.0$. In particular, the mass-to-flux ratios for all protostars with a standard hourglass magnetic field morphology are lower than 3.0. However, these ratios do not account for the contribution of the protostellar mass, which means they are likely significantly underestimated.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.15067-b31b1b.svg)](https://arxiv.org/abs/2503.15067) | **Exploring magnetised galactic outflows in starburst dwarf galaxies NGC 3125 and IC 4662**  |
|| S. Taziaux, et al. -- incl., <mark>A. Müller</mark> |
|*Appeared on*| *2025-03-20*|
|*Comments*| *19 pages, 16 figures*|
|**Abstract**|            The study of radio emission in starburst dwarf galaxies provides a unique opportunity to investigate the mechanisms responsible for the amplification and transport of magnetic fields. Local dwarfs are often considered proxies for early-Universe galaxies, so this study may provide insights into the role of non-thermal components in the formation and evolution of larger galaxies. By investigating the radio continuum spectra and maps of the starburst dwarf galaxies, we aim to draw conclusions on their magnetic field strengths and configurations, as well as the dynamics of cosmic ray (CR) transport. We perform a radio continuum polarimetry study of two of the brightest starburst IRAS Revised Bright Galaxy Sample (RBGS) dwarf galaxies, NGC 3125 and IC 4662. By combining data of the Australian Telescope Compact Array (2.1 GHz) and MeerKAT (1.28 GHz), we analyse the underlying emission mechanism and the CR transport in these systems. We find flat spectra in those dwarf galaxies over the entire investigated frequency range, which sharply contrasts with observations of massive spiral galaxies. Because the expected cooling time of CR electrons is much shorter than their escape time, we would expect a steepened steady-state CR electron spectrum. The flat observed spectra suggest a substantial contribution from free-free emission at high frequencies and absorption at low frequencies, which may solve this puzzle. For NGC 3125, we measure a degree of polarisation between 0.75% and 2.6%, implying a turbulent field and supporting the picture of a comparably large thermal emission component that could be sourced by stellar radiation feedback and supernovae.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.15306-b31b1b.svg)](https://arxiv.org/abs/2503.15306) | **Euclid Quick Data Release (Q1). Photometric redshifts and physical properties of galaxies through the PHZ processing function**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-03-20*|
|*Comments*| *Paper submitted as part of the A&A Special Issue `Euclid Quick Data Release (Q1)', 23 pages, 18 figures*|
|**Abstract**|            The ESA Euclid mission will measure the photometric redshifts of billions of galaxies in order to provide an accurate 3D view of the Universe at optical and near-infrared wavelengths. Photometric redshifts are determined by the PHZ processing function on the basis of the multi-wavelength photometry of Euclid and ground-based observations. In this paper, we describe the PHZ processing used for the Euclid Quick Data Release, the output products, and their validation. The PHZ pipeline is responsible for the following main tasks: source classification into star, galaxy, and QSO classes based on photometric colours; determination of photometric redshifts and of physical properties of galaxies. The classification is able to provide a star sample with a high level of purity, a highly complete galaxy sample, and reliable probabilities of belonging to those classes. The identification of QSOs is more problematic: photometric information seems to be insufficient to accurately separate QSOs from galaxies. The performance of the pipeline in the determination of photometric redshifts has been tested using the COSMOS2020 catalogue and a large sample of spectroscopic redshifts. The results are in line with expectations: the precision of the estimates are compatible with Euclid requirements, while, as expected, a bias correction is needed to achieve the accuracy level required for the cosmological probes. Finally, the pipeline provides reliable estimates of the physical properties of galaxies, in good agreement with findings from the COSMOS2020 catalogue, except for an unrealistically large fraction of very young galaxies with very high specific star-formation rates. The application of appropriate priors is, however, sufficient to obtain reliable physical properties for those problematic objects. We present several areas for improvement for future Euclid data releases.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Unable to get page count.
Syntax Warning: May not be a PDF file (continuing anyway)
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't read xref table
</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2503.15302/./figures/SEFphotometry.png', 'tmp_2503.15302/./figures/EXTQ1dataReleaseFigures/summary_Q1_DES.png', 'tmp_2503.15302/./figures/EXTQ1dataReleaseFigures/summary_Q1_EDFN.png', 'tmp_2503.15302/./figures/EDF-2panels.png']
copying  tmp_2503.15302/./figures/SEFphotometry.png to _build/html/
copying  tmp_2503.15302/./figures/EXTQ1dataReleaseFigures/summary_Q1_DES.png to _build/html/
copying  tmp_2503.15302/./figures/EXTQ1dataReleaseFigures/summary_Q1_EDFN.png to _build/html/
copying  tmp_2503.15302/./figures/EDF-2panels.png to _build/html/
exported in  _build/html/2503.15302.md
    + _build/html/tmp_2503.15302/./figures/SEFphotometry.png
    + _build/html/tmp_2503.15302/./figures/EXTQ1dataReleaseFigures/summary_Q1_DES.png
    + _build/html/tmp_2503.15302/./figures/EXTQ1dataReleaseFigures/summary_Q1_EDFN.png
    + _build/html/tmp_2503.15302/./figures/EDF-2panels.png
found figures ['tmp_2503.15303/./figures/VIS_WCS_quad_to_mm.py.png', 'tmp_2503.15303/./figures/NL_C

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\DS}[1]{ {\color{violet}(Douglas: #1)}}$
$\newcommand{\pz}{\phantom{0}}$
$\newcommand{\ps}{\phantom{-}}$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# $\Euclid$ Quick Data Release (Q1): Data release overview

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.15302-b31b1b.svg)](https://arxiv.org/abs/2503.15302)<mark>Appeared on: 2025-03-20</mark> -  _27 pages, 12 figures, data release at this https URL paper submitted to the special A&A issue_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>M. Schirmer</mark>, <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** The first Euclid Quick Data Release, Q1, comprises 63.1 deg $^2$ of the Euclid Deep Fields (EDFs) to nominal wide-survey depth. It encompasses visible and near-infrared space-based imaging and spectroscopic data, ground-based photometry in the $u$ , $g$ , $r$ , $i$ , and $z$ bands, as well as corresponding masks. Overall, Q1 contains about 30 million objects in three areas near the ecliptic poles around the EDF-North and EDF-South, as well as the EDF-Fornax field in the constellation of the same name. The purpose of this data release -- and its associated technical papers -- is twofold. First, it is meant to inform the community of the enormous potential of the $\Euclid$ survey data, to describe what is contained in these data, and to help prepare expectations for the forthcoming first major data release DR1. Second, itenables a wide range of initial scientific projects with wide-survey $\Euclid$ data, ranging from the early Universe to the Solar System. The Q1 data were processed with early versions of the processing pipelines, which already demonstrate good performance, with numerous improvements in implementation compared to pre-launch development. In this paper, we describe the sky areas released in Q1, the observations, a top-level view of the data processing of $\Euclid$ and associated external data, the Q1 photometric masks, and how to access the data. We also give an overview of initial scientific results obtained using the Q1 data set by Euclid Consortium scientists, and conclude with important caveats when using the data. As a complementary product, Q1 also contains observations of a star-forming area in Lynd's Dark Nebula 1641 in the Orion A Cloud, observed for technical purposes during $\Euclid$ 's performance-verification phase. This is a unique target, of a type not commonly found in $\Euclid$ 's nominal sky survey.

</div>

<div id="div_fig1">

<img src="tmp_2503.15302/./figures/SEFphotometry.png" alt="Fig6" width="100%"/>

**Figure 6. -** Photometric quality of the DES (left) and UNIONS (right) \acp{SEF} after calibration using _Gaia_ spectrophotometry and the appropriate survey bandpass, assumed to be independent of focal plane location for Q1.  Shown in each panel is the median offset (solid line) with respect to the _Gaia_ calibrators, as well as the \ac{NMAD} scatter (dashed line).  (*fig:SEFphotometry*)

</div>
<div id="div_fig2">

<img src="tmp_2503.15302/./figures/EXTQ1dataReleaseFigures/summary_Q1_DES.png" alt="Fig10.1" width="50%"/><img src="tmp_2503.15302/./figures/EXTQ1dataReleaseFigures/summary_Q1_EDFN.png" alt="Fig10.2" width="50%"/>

**Figure 10. -** Colour offsets.  Each panel here shows the spatial variation ($\partial_{\alpha,
    \delta}$) on scales of \ang{0.3;;} of the best-fit colour offset
    ($\Delta_{\rm color}$) for two of the \acp{EDF}. This
    statistic was determined for $g-r$(left), $r-i$(middle), and $i-z$(right),
    relative to the average stellar colour distribution in the \ac{EDF-S} area
    (top), and \ac{EDF-N}(bottom). In the case of \ac{EDF-S} we also show this
    result for DES DR2 data as the dashed dark histogram. The vertical black
    dotted line indicates the end-of-mission requirement. (*fig:Q1colorHomogeneityDECAMhistograms*)

</div>
<div id="div_fig3">

<img src="tmp_2503.15302/./figures/EDF-2panels.png" alt="Fig12" width="100%"/>

**Figure 12. -** Footprints of the Q1 visits to the \acp{EDF} shown on the sky in ecliptic coordinates overlaid on two 25$^\circ$$\times$ 55$^\circ$ cut-outs of a full-sky Digitized Sky Survey 2 optical image generated by the \ac{CDSdata}.
    _Left_: A strip of the northern ecliptic hemisphere down to $+45^\circ$, showing \ac{EDF-N}(blue) at the ecliptic pole in the Draco constellation; one of its fields is noticeably out of the regular pattern in order to avoid the bright 42 Dra star. _Right_: A strip of the southern ecliptic hemisphere down to $-45^\circ$ showing \ac{EDF-F}(red) in the Fornax constellation and \ac{EDF-S}(orange) in the Dorado constellation.  The Large Magellanic Cloud is noticeable close to the ecliptic pole. (*fig:EDFs*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.15302"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\fnurl}[2]{#1\footnote{\href{#2}{\url{#2}}}}$
$\newcommand{\orcid}[1]$
$\newcommand{\fnurl}[2]{#1\footnote{\href{#2}{\url{#2}}}}$</div>



<div id="title">

# $\Euclid$ Quick Data Release (Q1): VIS processing and data products

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.15303-b31b1b.svg)](https://arxiv.org/abs/2503.15303)<mark>Appeared on: 2025-03-20</mark> -  _Paper submitted as part of the A&A Special Issue `Euclid Quick Data Release (Q1)', 21 pages, 21 figures_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Paterson</mark>, <mark>M. Schirmer</mark>

</div>
<div id="abstract">

**Abstract:** This paper describes the $\ac{VIS PF}$ of the $\Euclid$ ground segment pipeline, which processes and calibrates raw data from the VIS camera. We present the algorithms used in each processing element, along with a description of the on-orbit performance of $\ac{VIS PF}$ , based on $\ac{PV}$ and Q1 data. We demonstrate that the principal performance metrics (image quality, astrometric accuracy, photometric calibration) are within pre-launch specifications. The image-to-image photometric scatter is less than $\SI{0.8}{\percent}$ and absolute astrometric accuracy compared to Gaia is 5 mas. Image quality is stable over all Q1 images with a $\ac{FWHM}$ of $\ang{;;0.16}$ . The stacked images (combining four nominal and two short exposures) reach $\IE=25.6$ ( $10 \sigma$ , measured as the variance of $\ang{;;1.3}$ diameter apertures). We also describe quality control metrics provided with each image and an appendix provides a detailed description of the provided data products. The excellent quality of these images demonstrates the immense potential of $\Euclid$ VIS data for weak lensing. VIS data, covering most of the extragalactic sky, will provide a lasting high-resolution atlas of the Universe.

</div>

<div id="div_fig1">

<img src="tmp_2503.15303/./figures/VIS_WCS_quad_to_mm.py.png" alt="Fig1" width="100%"/>

**Figure 1. -** The layout of quadrants in the \ac{FPA}. The detectors, each containing four quadrants, are arranged in a \(6\times6\) array. The mean \ac{CCD} spacing is \SI{1.53 \pm 0.03}{\milli\meter} and \SI{7.74 \pm 0.06}{\milli\meter} in the \(x\)- and \(y\)-directions, respectively. (*fig:focal-plane-layout*)

</div>
<div id="div_fig2">

<img src="tmp_2503.15303/./figures/NL_CCD62_OUVISPaperQ1_NLCorr_noGrid_V3.png" alt="Fig3" width="100%"/>

**Figure 3. -** Pre-launch measurements of VIS nonlinearity based on ground testing for \ac{CCD} 6-2. Each of the four lines corresponds to a separate quadrant. _Top panel_: measured intensity as a function of pulse time width. _Bottom panel_: linear signal divided by measured signal. (*fig:nonlinearity*)

</div>
<div id="div_fig3">

<img src="tmp_2503.15303/./figures/mumax_bfe.png" alt="Fig5" width="100%"/>

**Figure 5. -** Effect of \ac{BFE} correction on the difference of the surface brightness of the brightest pixel of stars in nominal versus short exposures as a function of VIS magnitude in an aperture of 13 pixels or \ang{;;1.3} diameter. (*fig:bfe-mumax*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.15303"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]{\orcidlink{#1}}$</div>



<div id="title">

# Euclid Quick Data Release (Q1): NIR processing and data products

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.15304-b31b1b.svg)](https://arxiv.org/abs/2503.15304)<mark>Appeared on: 2025-03-20</mark> -  _24 pages, 18 figures_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>, <mark>K. Paterson</mark>, <mark>M. Schirmer</mark>

</div>
<div id="abstract">

**Abstract:** This paper describes the $\ac{NIR PF}$ that processes near-infrared images from the $\ac{NISP}$ instrument onboard the $\Euclid$ satellite. $\ac{NIR PF}$ consists of three main components: (i) a common pre-processing stage for both photometric (NIR) and spectroscopic (SIR) data to remove instrumental effects; (ii) astrometric and photometric calibration of NIR data, along with catalogue extraction; and (iii) resampling and stacking. The necessary calibration products are generated using dedicated pipelines that process observations from both the early $\ac{PV}$ phase in 2023 and the nominal survey operations. After outlining the pipeline’s structure and algorithms, we demonstrate its application to $\Euclid$ Q1 images. For Q1, we achieve an astrometric accuracy of 9--15 mas, a relative photometric accuracy of 5 mmag, and an absolute flux calibration limited by the 1 \% uncertainty of the $\ac{HST}$ CALSPEC database. We characterise the $\ac{PSF}$ that we find very stable across the focal plane, and we discuss current limitations of $\ac{NIR PF}$ that will be improved upon for future data releases.

</div>

<div id="div_fig1">

<img src="tmp_2503.15304/./figures/histogram_mag_psf_vs_mag_aper_flux_radius_F006_3800_NIR_Y.png" alt="Fig15.1" width="33%"/><img src="tmp_2503.15304/./figures/histogram_mag_psf_vs_mag_aper_flux_radius_F006_3800_NIR_J.png" alt="Fig15.2" width="33%"/><img src="tmp_2503.15304/./figures/histogram_mag_psf_vs_mag_aper_flux_radius_F006_3800_NIR_H.png" alt="Fig15.3" width="33%"/>

**Figure 15. -** Comparison between \ac{PSF}-fitting and aperture photometry.
_ Left column:_ Histograms of the half-light diameter for the three NISP passbands. The dashed lines indicate the ranges of point-sources plotted in the right panels and used by {\tt PSFex} for \ac{PSF} modelling. _ Right column:_ Difference between \texttt{MAG\_PSF} and \texttt{MAG\_APER}, colour-coded according to the half-light diameter. The mean difference and the slope of the linear regression (black line) for magnitudes between 17 and 21 (dashed lines) are also shown. (*figure:hist_mag_psf_aper*)

</div>
<div id="div_fig2">

<img src="tmp_2503.15304/./figures/Baseline_DET13_example_rev.png" alt="Fig2" width="100%"/>

**Figure 2. -** Example of the science pixels' baseline distribution for one of the detectors (DET13) measured during \ac{PV}. Solid black vertical lines show the boundary of the \ac{DNL} range defined during the
ground-test campaign. (*fig:baseline*)

</div>
<div id="div_fig3">

<img src="tmp_2503.15304/./figures/chi2_data_pdf_dof13.png" alt="Fig4" width="100%"/>

**Figure 4. -** Comparison between the \ac{PDF} of the pixel quality factors ($\chi^2$) for a spectroscopic MACC(15,16,11) exposure for one of the NISP detectors (blue histogram), and the \ac{PDF} calculated for a $\chi^2$ distribution with
${N_{\rm dof}}=13$(red line). The threshold $\chi^2=50$(dashed vertical line) distinguishes good- from bad-quality pixels. The $\chi^2$ value is encoded on-board as an 8-bit integer and thus has an upper limit of 255. Hence, the last bin corresponding to $\chi^2=255$ includes not only \acp{CR}, but all other pixels with a poor slope fit, such as bad pixels. (*figure:chi2*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.15304"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\eq}{Eq{.}~}$
$\newcommand{\eqs}{Eqs{.}~}$
$\newcommand{\tab}{Tab{.}~}$
$\newcommand{\fg}{Fig{.}~}$
$\newcommand{\fgs}{Figs{.}~}$
$\newcommand{\sct}{Sect{.}~}$
$\newcommand{\scts}{Sects{.}~}$
$\newcommand{\col}{Col{.}}$
$\newcommand{\cols}{Cols{.}}$
$\newcommand$
$\newcommand$
$\newcommand{\lo}[2]{\noindent{\color{royalblue} \bf[ \triangle \st{#1} #2]}}$
$\newcommand{\vl}[2]{\noindent{\color{violet} \bf[ VL / \st{#1} #2]}}$
$\newcommand{\dd}[2]{\noindent{\color{green} \bf[ DD / \st{#1} #2]}}$
$\newcommand{\rr}[2]{\noindent{\color{darkred} \bf[ RR / \st{#1} #2]}}$
$\newcommand{\id}[2]{\noindent{\color{orange} \bf[ ID / \st{#1} #2]}}$
$\newcommand{\apa}[2]{\noindent{\color{pink} \bf[ APA / \st{#1} #2]}}$
$\newcommand{\afm}[2]{\noindent{\color{grey} \bf[ AFM / \st{#1} #2]}}$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# Euclid Quick Data Release (Q1): Exploring galaxy morphology across cosmic time through Sérsic fits

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.15309-b31b1b.svg)](https://arxiv.org/abs/2503.15309)<mark>Appeared on: 2025-03-20</mark> -  _Paper submitted as part of the A&A Special Issue `Euclid Quick Data Release (Q1)', 23 pages, 16 figures_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** We present the results of the single-component Sérsic profile fitting for the magnitude-limited sample of $\IE$ $<23$ galaxies within the 63.1 deg $^2$ area of the Euclid Quick Data Release (Q1). The associated morphological catalogue includes two sets of structural parameters fitted using \texttt{SourceXtractor++} : one for VIS $\IE$ images and one for a combination of three NISP images in $\YE$ , $\JE$ and $\HE$ bands. We compare the resulting Sérsic parameters to other morphological measurements provided in the Q1 data release, and to the equivalent parameters based on higher-resolution $\HST$ imaging. These comparisons confirm the consistency and the reliability of the fits to Q1 data. Our analysis of colour gradients shows that NISP profiles have systematically smaller effective radii ( $R_{\rm e}$ ) and larger Sérsic indices ( $n$ ) than in VIS. In addition, we highlight trends in NISP-to-VIS parameter ratios with both magnitude and $n_{\rm VIS}$ . From the 2D bimodality of the $(u-r)$ colour- $\log(n)$ plane, we define a $(u-r)_{\rm lim}(n)$ that separates early- and late-type galaxies (ETGs and LTGs). We use the two subpopulations to examine the variations of $n$ across well-known scaling relations at $z<1$ . ETGs display a steeper size--stellar mass relation than LTGs, indicating a difference in the main drivers of their mass assembly. Similarly, LTGs and ETGs occupy different parts of the stellar mass --  star-formation rate plane, with ETGs at higher masses than LTGs, and further down below the Main Sequence of star-forming galaxies. This clear separation highlights the link known between the shutdown of star formation and morphological transformations in the $\Euclid$ imaging data set. In conclusion, our analysis demonstrates both the robustness of the Sérsic fits available in the Q1 morphological catalogue and the wealth of information they provide for studies of galaxy evolution with $\Euclid$ .

</div>

<div id="div_fig1">

<img src="tmp_2503.15309/./Figures/NISP_vs_VIS_R_e_mag_color_n_2Dhist.png" alt="Fig9" width="100%"/>

**Figure 9. -** The ratio between effective radius based on the common Sérsic model for three NISP images ($R_\mathrm{e,  NISP}$) and that from the Sérsic fit to the VIS image ($R_\mathrm{e,  VIS}$) as a function of isophotal VIS magnitude. The four panels correspond to the redshift bins we use to trace evolutionary trends in $\sct$ \ref{sc:results-evol-plots}. The 2-dimensional (2D) histogram in each panel are colour-coded by the median $n_\mathrm{VIS}$ for galaxies in a given $[\Delta m_{\rm ISO} (\IE)=0.5$, $\Delta$\lo$g(R_\mathrm{e,  NISP}/R_\mathrm{e,  VIS})=0.05]$ pixel. The 2D histogram includes pixels with $N>100$ galaxies. In more sparsely populated regions we instead display individual galaxies colour-coded by their $n_\mathrm{VIS}$. The box-and-whisker diagrams show median values (red solid lines), interquartile range (IQR, boxes) and the extent of the 1.5 IQR range above the 75th and below the 25th percentile (whiskers) for $R_\mathrm{e,  NISP}/R_\mathrm{e,  VIS}$ in $\Delta m_{\rm ISO}=0.5$ bins of isophotal \IE. The grey dashed line corresponds to the constant ratio of $R_\mathrm{e,  NISP}/R_\mathrm{e,  VIS}=1$. (*f7*)

</div>
<div id="div_fig2">

<img src="tmp_2503.15309/./Figures/sersic_index_histogram_percent_bins_ML_smooth_or_featured.png" alt="Fig7.1" width="25%"/><img src="tmp_2503.15309/./Figures/axis_ratio_histogram_percent_bins_ML_edge_on.png" alt="Fig7.2" width="25%"/><img src="tmp_2503.15309/./Figures/sersic_index_histogram_percent_bins_ML_bulge_size.png" alt="Fig7.3" width="25%"/><img src="tmp_2503.15309/./Figures/axis_ratio_histogram_percent_bins_ML_edge_on_bulge.png" alt="Fig7.4" width="25%"/>

**Figure 7. -** Distribution of Sérsic parameters measured on the VIS image for sub-samples of galaxies depending on their classification by the \texttt{Zoobot} labels, with different line styles corresponding to the different shares of expected votes predicted by \texttt{Zoobot} to consider that a galaxy falls into a certain category. Sample sizes are systematically indicated in the legend, and a 50\% threshold of voters is adopted when not specified (bottom plots). _Top-left_: distribution of $n$ for galaxies classified as either smooth (red lines) or displaying a disc or features (blue lines) shows a clear dichotomy with the latter sample being almost exclusively made of galaxies with $n\sim1$, whereas the former contains a lot of galaxies with high values of $n$ and increasingly so for higher selection threshold. _Top-right_: distribution of $q$ for disc galaxies depending if they are edge-on or not. Again, a clear bimodality appears, with low $q$ for edge-on galaxies and higher values spread across a wider interval for non-edge-on, hence from face-on to moderately inclined galaxies. Using higher thresholds reinforces this dichotomy. _Bottom-left_: Distribution for disc or featured galaxies of $n$ for different categories of \texttt{Zoobot} bulge sizes. Histograms shift to higher values of $n$ for larger relative bulge sizes. _Bottom-right_: Distribution of $q$ for edge-on disc galaxies, depending on if they display no bulge, a boxy bulge, or a rounded bulge. The presence of a bulge, either rounded or boxy, leads to higher $q$.
 (*fig:sersic-vs-zoobot*)

</div>
<div id="div_fig3">

<img src="tmp_2503.15309/./Figures/sfr_vs_mass_median_redshift_bins_nsersic_stats.png" alt="Fig14" width="100%"/>

**Figure 14. -** SFR versus $M_\ast$ diagrams for the four consecutive intervals of redshift, colour-coded by the median $n$(top row) and associated dispersion around it (bottom row) in each cell of the plane. We over-plotted density contours at the 10\%, 50\%, and 90\% levels, as well as the 90\% mass completeness limit for the ETG and LTG samples (red and blue vertical dashed lines, respectively). The SFMS from \cite{Speagle-2014-Main-Sequence} and \cite{Popesso-2023-Main-Sequence} are displayed as black dashed and dotted lines, respectively, whereas a fit to the $M_\ast$--SFR relation of our sample of LTG galaxies is shown as a black solid line.
 (*fig:sfr-mass*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.15309"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Mstarsun}{\logten(M_{\ast}/M_\odot)}$
$\newcommand{\citenopar}[1]{$
$  \begingroup$
$  \@tempcnta\z@$
$  \@for\@II:=#1\do{\advance\@tempcnta\@ne}$
$  \@tempcntb\z@$
$  \@for\@II:=#1\do{$
$    \advance\@tempcntb\@ne$
$    \citeauthor{\@II} \citeyear{\@II}$
$    \ifnum\@tempcntb<\@tempcnta, \fi$
$  }$
$  \endgroup$
$}$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# Euclid Quick Data Release (Q1): Exploring galaxy properties with a multi-modal foundation model

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.15312-b31b1b.svg)](https://arxiv.org/abs/2503.15312)<mark>Appeared on: 2025-03-20</mark> -  _Paper submitted as part of the A&A Special Issue `Euclid Quick Data Release (Q1)', 31 pages, 17 figures_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** Modern astronomical surveys, such as the $\Euclid$ mission, produce high-dimensional, multi-modal data sets that include imaging and spectroscopic information for millions of galaxies. These data serve as an ideal benchmark for large, pre-trained multi-modal models, which can leverage vast amounts of unlabelled data. In this work, we present the first exploration of $\Euclid$ data with \texttt{AstroPT} , an autoregressive multi-modal foundation model trained on approximately $\(300 000\)$ optical and infrared $\Euclid$ images and spectral energy distributions (SEDs) from the first $\Euclid$ Quick Data Release. We compare self-supervised pre-training with baseline fully supervised training across several tasks: galaxy morphology classification; redshift estimation; similarity searches; and outlier detection. Our results show that: (a) \texttt{AstroPT} embeddings are highly informative, correlating with morphology and effectively isolating outliers; (b) including infrared data helps to isolate stars, but degrades the identification of edge-on galaxies, which are better captured by optical images; (c) simple fine-tuning of these embeddings for photometric redshift and stellar mass estimation outperforms a fully supervised approach, even when using only $\(1\%\)$ of the training labels; and (d) incorporating SED data into \texttt{AstroPT} via a straightforward multi-modal token-chaining method improves photo- $\(z\)$ predictions, and allow us to identify potentially more interesting anomalies (such as ringed or interacting galaxies) compared to a model pre-trained solely on imaging data.

</div>

<div id="div_fig1">

<img src="tmp_2503.15312/./fig/Euclid_image.png" alt="Fig13.1" width="50%"/><img src="tmp_2503.15312/./fig/diagram.png" alt="Fig13.2" width="50%"/>

**Figure 13. -** {\tt AstroPT's} training objective is predicting the next patch in a sequence of galaxy image (and other modalities') patches. On the left we show the order of galaxy patches used in this work, and on the right we show how each patch is processed by {\tt AstroPT}. (*fig_galaxyastropt*)

</div>
<div id="div_fig2">

<img src="tmp_2503.15312/./fig/Smooth_emb.png" alt="Fig16.1" width="50%"/><img src="tmp_2503.15312/./fig/Spiral_arms_emb.png" alt="Fig16.2" width="50%"/>

**Figure 16. -** UMAP visualisations of the self-supervised embeddings from {\tt AstroPT} trained on VIS (left), VIS+NISP (centre), and VIS+NISP+SED (right) coloured by morphological parameters derived from the GZ catalogue \citep{Q1-SP047}. Representations show clear clustering of smooth, non-spiral, and non-smooth, spiral galaxies. (*fig:UMAP_visualisation_morphology_spiral*)

</div>
<div id="div_fig3">

<img src="tmp_2503.15312/./fig/disk_edgeon1a.png" alt="Fig17" width="100%"/>

**Figure 17. -** UMAP visualisations of the self-supervised embeddings from {\tt AstroPt} coloured by morphological parameters derived from the GZ catalogue \citep{Q1-SP047}. $Left$: Representations from VIS images alone, showing a distinct region for edge-on galaxies. $Middle$: Representations incorporating both VIS and NISP data, where the separation of edge-on galaxies becomes less distinct. $Right$: Representations incorporating VIS, NISP, and SED data. $Bottom$: Example cutouts and SEDs of the edge-on galaxies. (*fig:UMAP_visualisation_morphology_edge_on*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.15312"></div>

...

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\VIS}{I_{\scriptscriptstyle\rm E} }$
$\newcommand{\VISdot}{I_{\scriptscriptstyle\rm E}}$
$\newcommand{\mstar}{\mbox{M_\star}\xspace}$
$\newcommand{\msunh}{\mbox{M_\odot/h}\xspace}$
$\newcommand{\msun}{\mbox{M_\odot}\xspace}$
$\newcommand{\halpha}{\mbox{H\alpha}\xspace}$
$\newcommand{\disperse}{\mbox{{\sc DisPerSE}}\xspace}$
$\newcommand{\deep}{\mbox{\mathcal{D}}\xspace}$
$\newcommand{\deepmass}{\mbox{\mathcal{D}_{M_\star}}\xspace}$
$\newcommand{\deepnoise}{\mbox{\mathcal{D}_{\rm noise}}\xspace}$
$\newcommand{\deepsamp}{\mbox{\mathcal{D}_{\rm 60\%}}\xspace}$
$\newcommand{\deepnoisesamp}{\mbox{\mathcal{D}_{\rm noise,60\%}}\xspace}$
$\newcommand{\deepwf}{\mbox{\mathcal{D}^{\rm w FoG}}\xspace}$
$\newcommand{\deepwof}{\mbox{\mathcal{D}^{\rm FoG,corr}}\xspace}$
$\newcommand{\deepnoisesampwf}{\mbox{\mathcal{D}^{\rm w FoG}_{\rm noise,60\%}}\xspace}$
$\newcommand{\deepnoisesampwof}{\mbox{\mathcal{D}^{\rm FoG,corr}_{\rm noise,60\%}}\xspace}$
$\newcommand{\deepmasssamp}{\mbox{\mathcal{D}_{M_\star,\rm 60\%}}\xspace}$
$\newcommand{\deepmassnoisesampwf}{\mbox{\mathcal{D}^{\rm w FoG}_{M_\star,\rm noise,60\%}}\xspace}$
$\newcommand{\deepmassnoisesampwof}{\mbox{\mathcal{D}^{\rm FoG,corr}_{M_\star,\rm noise,60\%}}\xspace}$
$\newcommand{\deepmasswf}{\mbox{\mathcal{D}^{\rm w FoG}_{M_\star}}\xspace}$
$\newcommand{\deepmasswof}{\mbox{\mathcal{D}^{\rm FoG,corr}_{M_\star}}\xspace}$
$\newcommand{\flag}{\mbox{Flagship}\xspace}$
$\newcommand{\gaea}{\mbox{GAEA}\xspace}$
$\newcommand{\laigle}[1]{{\color{pink}{ \textbf\small{#1}}}}$
$\newcommand{\Kat}[1]{{\color{applegreen}{{\bf #1}}}}$
$\newcommand{\Flo}[1]{{\color{magenta}{{[FD: \bf #1]}}}}$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# Euclid Quick Data Release (Q1): Galaxy shapes and alignments in the cosmic web

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.15333-b31b1b.svg)](https://arxiv.org/abs/2503.15333)<mark>Appeared on: 2025-03-20</mark> -  _Paper submitted as part of the A&A Special Issue `Euclid Quick Data Release (Q1)', 22 pages, 19 figures_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** Galaxy morphologies and shape orientations are expected to correlate with their large-scale environment, since they grow by accreting matter from the cosmic web and are subject to interactions with other galaxies. Cosmic filaments are extracted in projection from the Euclid Quick Data Release 1 (covering 63.1 $\mathrm{deg}^2$ ) at $0.5<z<0.9$ in tomographic slices of 170 comoving $h^{-1}\mathrm{Mpc}$ using photometric redshifts. Galaxy morphologies are accurately retrieved thanks to the excellent resolution of VIS data. The distribution of massive galaxies ( $M_* > 10^{10} M_\odot$ ) in the projected cosmic web is analysed as a function of morphology measured from VIS data. Specifically, the 2D alignment of galaxy shapes with large-scale filaments is quantified as a function of Sérsic indices and masses. We find the known trend that more massive galaxies are closer to filament spines. At fixed stellar masses, morphologies correlate both with densities and  distances to large-scale filaments. In addition, the large volume of this data set allows us to detect a signal indicating that there is a preferential alignment of the major axis of massive early-type galaxies along projected cosmic filaments. Overall, these results demonstrate our capabilities to carry out detailed studies of galaxy environments with $\Euclid$ , which will be extended to higher redshift and lower stellar masses with the future Euclid Deep Survey.

</div>

<div id="div_fig1">

<img src="tmp_2503.15333/./figs/bimodality_angle_err_.png" alt="Fig2" width="100%"/>

**Figure 2. -** Sérsic index versus mass distribution for galaxies in the selected-sample, colour-coded by the median angle between galaxy major axes and the closest filament $\theta_{\rm fil}$. All three fields are combined here and only galaxies with a median distance to filament smaller than 2$\Delta_{\rm skl}$ are kept. $\Delta_{\rm skl}$ is the mean uncertainty on filament position (see Sect. \ref{sec:tool}).
   (*fig:2dalign*)

</div>
<div id="div_fig2">

<img src="tmp_2503.15333/./figs/errz.png" alt="Fig4" width="100%"/>

**Figure 4. -** *Top panels*: smallest redshift interval containing $70$\% of the probability derived from the {\tt Phosphoros} PDF($z$) as a function of redshift and masses for galaxies in EDF-N (left), EDF-S (middle), and EDF-F (right), as described in Sect. \ref{sec:reldes}. In each redshift bin, the markers indicate the mean of the $70$\%$z$-interval, and the error bars indicate the 16th and 84th percentiles. A $5 \sigma$ clipping is applied before computing these statistics. Because we rely on the {\tt NNPZ} stellar masses to select galaxies, we also impose for consistency that the  {\tt NNPZ} median redshift fall in the $70$\%$z$-interval.  *Bottom panels*: the corresponding uncertainty in $h^{-1}{\rm Mpc}$, which characterises the typical confusion length of galaxies along the line of sight. The black dashed line indicates the thickness for tomographic slices that we consider in this study, calibrated to encompass photometric redshift errors of galaxies more massive than $10^{10.3}M_\odot$ at $0.5<z<0.9$. The grey shaded area indicates the redshift range considered in this study.
 (*fig:confusion_length*)

</div>
<div id="div_fig3">

<img src="tmp_2503.15333/./figs/slice_dtfe_fil_1_wgal_all_4.png" alt="Fig5" width="100%"/>

**Figure 5. -** Zoom into the slice from the EDF-F at $z=0.55$ with three selected galaxy cutouts from the MER mosaic (left subpanels), with their positions in the slice marked with coloured squares. In the top right corner of each cutout, we displayed the orientation of the closest segment in each of the realisations (the distance to the galaxy on the stamp is not representative of the real distance). The coordinates of the galaxies are indicated in the bottom of the cutout. The slice thickness is $170$ $h^{-1}{\rm Mpc}$. One hundred realisations from the \texttt{DisPerSE} skeleton extracted in 2D are overplotted, color-coded by $\log_{10}\left[\langle{\rm dtfe}\rangle/\sigma_{\rm dtfe}\right]$, which is the density estimation from the Delaunay tesselation (in one of the realisations) in the background (normalised by the standard deviation). Dots represent galaxies in one realisation of the slice, with their size scaling with galaxy masses.  The subpanels in the two middle columns display the distribution of galaxy projected angles and distances between the galaxy main axis and the closest filament in each of the 100 realisations. We made use of the ESA Datalabs resources  (Datalabscite)  to extract galaxy stamps shown here.
 (*fig:method*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.15333"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]$
$\newcommand{\tmax}{{\vartheta_{\rm max}}}$
$\newcommand{\refer}[1]{,\ #1; [{\tt #1}]}$
$\newcommand{\itemm}{$
$\medskip\noindent}$
$\newcommand{\pd}{\phantom{1}}$</div>



<div id="title">

# $\Euclid$\/: Title of your paper$\thanks{This paper is published on     behalf of the Euclid Consortium}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.15334-b31b1b.svg)](https://arxiv.org/abs/2503.15334)<mark>Appeared on: 2025-03-20</mark> -  _Paper submitted as part of the A&A Special Issue `Euclid Quick Data Release (Q1), 27 pages, 10 figures_

</div>
<div id="authors">

C. Duffy, et al. -- incl., <mark>K. Paterson</mark>, <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** ${\color{blue} \bf NEW VERSION: We have a modified style file,     aaEC.cls, used here; with that, the bug reported in Sect. 8 no     longer occurs. In order to demonstrate this, the long author list     of EROData was copied here.}$ \ This is a template file for the preparation of Euclid Consortium (EC) papersfor submission to A \& A. The list of lead authors and affiliations shouldbe prepared by the authors only on first submission to the ECEditorial Board (ECEB). Once thepaper is circulated within the full EC, and eligible authors can signup, then you should instead use the author and affiliation listgenerated for your paper on the Publication Portal; any editing ofthese lists byhand is error prone and must not be done.With the macros \textbackslash{\tt titlerunning} and \textbackslash{\tt authorrunning} ,shortened versions of the author list and the title should be given, suchthat they fit into the headline of each odd-numbered page.As a default, it is recommended to use the "traditional" abstract, asgiven in this example here. Nevertheless, the structure of theabstract should follow the journal's guidelines, i.e., context, aims,methods, and results, just without explicitly writing the headings.In the Appendix, we provide a list of EC publications, which will becontinuously updated. Furthermore, we have prepared the file ${\tt  Euclid.bib}$ , which contains most of the EC papers and can be usedfor preparing your reference list.

</div>

<div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.15334"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]{\orcidlink{#1}}$
$\newcommand{\av}{A_V}$
$\newcommand{\zsun}{Z_\odot}$
$\newcommand{\zzsun}{Z/Z_\odot}$
$\newcommand{\hi}{H{\sc i}}$
$\newcommand{\hii}{H{\sc ii}}$
$\newcommand{\nint}{\dot{N}_\mathrm{int}}$
$\newcommand{\ngas}{\dot{N}_\mathrm{gas}}$
$\newcommand{\ndust}{\dot{N}_\mathrm{dust}}$
$\newcommand{\mhi}{M_{HI}}$
$\newcommand{\htwo}{H_2}$
$\newcommand{\Nhtwo}{N_{H2}}$
$\newcommand{\mhtwo}{M_{H2}}$
$\newcommand{\mstar}{M_*}$
$\newcommand{\mgas}{M_{\rm gas}}$
$\newcommand{\msun}{M_\odot}$
$\newcommand{\aco}{\alpha_{CO}}$
$\newcommand{\logoh}{12+log(O/H)}$
$\newcommand{\cmtwo}{cm^{-2}}$
$\newcommand{\hst}{{\it HST}}$
$\newcommand{\jwst}{{\it JWST}}$
$\newcommand{\spit}{\textit{Spitzer}}$
$\newcommand{\hers}{\textit{Herschel}}$
$\newcommand{\lsun}{L_\odot}$
$\newcommand{\lcdm}{\LambdaCDM}$
$\newcommand{\magarc}{mag arcsec^{-2}}$
$\newcommand{\TS}[1]{ {\color{red}\it #1}}$
$\newcommand{\om}[1]{ {\color{cyan} #1}}$
$\newcommand{\afm}[1]{ {\color{green} #1}}$
$\newcommand{\pd}{\phantom{1}}$
$\newcommand{\pd}{\phantom{1}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\arraystretch}{1.4}$
$\newcommand{\arraystretch}{1.4}$</div>



<div id="title">

# $\Euclid$: Quick Data Release (Q1) -- A census of dwarf galaxies across a range of distances and environments$\thanks{This paper is published on behalf of the Euclid Consortium.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.15335-b31b1b.svg)](https://arxiv.org/abs/2503.15335)<mark>Appeared on: 2025-03-20</mark> -  _Paper submitted as part of the A&A Special Issue `Euclid Quick Data Release (Q1)', 27 pages, 19 figures, 5 tables_

</div>
<div id="authors">

F. R. Marleau, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** The $\Euclid$ Q1 fields were selected for calibration purposes in cosmology and are therefore relatively devoid of nearby galaxies. However, this is precisely what makes them interesting fields in which to search for dwarf galaxies in local density environments. We take advantage of the unprecedented depth, spatial resolution, and field of view of the $\Euclid$ Quick Release (Q1) to build a census of dwarf galaxies in these regions. We have identified dwarf galaxies in a representative sample of 25 contiguous tiles in the Euclid Deep Field North (EDF-N), covering an area of 14.25 $\deg^2$ . The dwarf galaxy candidates were identified using a semi-automatic detection method, based on properties measured by the $\Euclid$ pipeline and released as part of the MER catalogue. A selection cut in surface brightness and magnitude was used to produce an initial dwarf candidate catalogue, followed by a cut in morphology (removing background spirals) and $\IE-\HE$ colour (removing red ellipticals). This catalogue was then visually classified to produce a final sample of dwarf candidates, including their morphology, number of nuclei, globular cluster (GC) richness, and presence of a blue compact centre. We identified 2674 dwarf candidates, corresponding to 188 dwarfs per square degree. The visual classification of the dwarfs reveals a slightly uneven morphological mix of 58 \% ellipticals and 42 \% irregulars, with very few potentially GC-rich (1.0 \% ) and nucleated (4.0 \% ) candidates but a noticeable fraction (6.9 \% ) of dwarfs with blue compact centres. The distance distribution of 388 (15 \% ) of the dwarf candidates with spectroscopic redshifts peaks at about 400 Mpc. Their stellar mass distribution confirms that our selection effectively identifies dwarfs while minimising contamination. The most prominent dwarf overdensities are dominated by dEs, while dIs are more evenly distributed across the field of view. This work highlights $\Euclid$ ’s remarkable ability to detect and characterise dwarf galaxies across diverse masses, distances, and environments.

</div>

<div id="div_fig1">

<img src="tmp_2503.15335/./figures/marleau-fig4.png" alt="Fig10" width="100%"/>

**Figure 10. -** _Left:_ Isodensity contour plot in average surface brightness versus apparent magnitude space for the visually identified dwarf galaxy candidates in the reference tile 102158889 with a score $\geq 0.8$. The dwarf candidate sample was manually cleaned to ensure good MER photometry of the sources. The upper portion of the outermost contour is used as basis for the subsequent automated selection cut (_right_). The non-dwarf galaxies (\texttt{POINT\_LIKE\_PROB}$< 0.8$"; grey points) and stars (\texttt{POINT\_LIKE\_PROB}$\geq 0.8$"; pink points) are also included for reference. _Right:_ An example of the automatic selection cut in the reference tile. We extended slightly the upper portion of the outer density contour (red) to select the few bright dwarf candidates with score $\geq 0.8$ that would otherwise have been missed. The final selection cut is shown in orange. Many of the less confident dwarfs (score $< 0.8$) are also selected by this line. (*fig:selection*)

</div>
<div id="div_fig2">

<img src="tmp_2503.15335/./figures/marleau-fig14.png" alt="Fig16" width="100%"/>

**Figure 16. -** Distribution of the properties of the Q1 dwarf candidates with spectroscopic redshift (pale blue-violet) overlaid on the properties of all galaxies in the MER catalogue (pale brown) and the automatically selected dwarf candidates with spectroscopic redshift in all 25 Q1 tiles (dark grey). The absolute magnitudes of the \Euclid ERO Perseus dwarfs from \citet{EROPerseusDGs}, at a distance of 72 Mpc, are shown for comparison (dark red). The Perseus dwarfs are closer than the majority of the dwarf candidates in the Q1 data and therefore are detected to fainter absolute magnitudes. The physical sizes of the Q1 and Perseus dwarf candidates (dark red) cover a similar range, assuming that the Perseus dwarfs have a semimajor axis approximately equal to twice their effective radius ($2  \times R_{\rm e}$). (*fig:specz*)

</div>
<div id="div_fig3">

<img src="tmp_2503.15335/./figures/marleau-fig17.png" alt="Fig18" width="100%"/>

**Figure 18. -** Spatial distribution of the final dwarf candidates is shown alongside the HECATE galaxies within the 25 tiles (marked as open black circles with a transparent white fill) and dwarf galaxies with measured spectroscopic redshifts (pale blue-violet filled squares). The colour map is based on the number of dwarfs within bins of RA and Dec, with this count normalised to a value of 1. According to the colour bar, orange-yellow colours indicate regions with dwarf galaxy overdensities. HECATE host galaxies and dwarfs with spectroscopic redshifts are displayed in the left and right panels, representing distances, $D$, in the ranges of $90 \leq D/{\rm Mpc} < 150$ and $150 \leq D/{\rm Mpc} < 210$, respectively. The white star indicates the location of the Cat's eye nebula where a lack of dwarfs is expected. (*fig:soverdensities_with_zspecs*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.15335"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\url}[1]{\href{#1}{#1}}$
$\newcommand{\dodoi}[1]{doi:~\href{http://doi.org/#1}{\nolinkurl{#1}}}$
$\newcommand{\doeprint}[1]{\href{http://ascl.net/#1}{\nolinkurl{http://ascl.net/#1}}}$
$\newcommand{\doarXiv}[1]{\href{https://arxiv.org/abs/#1}{\nolinkurl{https://arxiv.org/abs/#1}}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand\natexlab{#1}$</div>



<div id="title">

# Refractory and Volatile Species in the UV-to-IR Transmission Spectrum of Ultra-hot Jupiter WASP-178b with HST and JWST

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.15472-b31b1b.svg)](https://arxiv.org/abs/2503.15472)<mark>Appeared on: 2025-03-20</mark> -  _34 pages, 16 figures, Accepted to AJ_

</div>
<div id="authors">

J. D. Lothringer, et al. -- incl., <mark>C. Gapp</mark>

</div>
<div id="abstract">

**Abstract:** $\noindent$ The atmospheres of ultra-hot Jupiters are unique compared to other planets because of the presence of both refractory and volatile gaseous species, enabling a new lens to constrain a planet's composition, chemistry, and formation. WASP-178b is one such ultra-hot Jupiter that was recently found to exhibit enormous NUV absorption between 0.2 and 0.4 $\mu$ m from some combination of Fe+, Mg, and SiO. Here, we present new infrared observations of WASP-178b with HST/WFC3 and JWST/NIRSpec/G395H, providing novel measurements of the volatile species $H_2$ O and CO in WASP-178b's atmosphere. Atmospheric retrievals find a range of compositional interpretations depending on which dataset is retrieved, the type of chemistry assumed, and the temperature structure parametrization used due to the combined effects of thermal dissociation, the lack of volatile spectral features besides $H_2$ O and CO, and the relative weakness of $H_2$ O and CO themselves. Taken together with a new state-of-the-art characterization of the host star, our retrieval analyses suggests a solar to super-solar [ O/H ] and [ Si/H ] , but sub-solar [ C/H ] , perhaps suggesting rock-laden atmospheric enrichment near the $H_2$ O iceline. To obtain meaningful abundance constraints for this planet, it was essential to combine the JWST IR data with short-wavelength HST observations, highlighting the ongoing synergy between the two facilities.

</div>

<div id="div_fig1">

<img src="tmp_2503.15472/./best_fit_compare_JWSTonly_v5.png" alt="Fig6.1" width="50%"/><img src="tmp_2503.15472/./best_fit_compare_full_6.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** Top: Best-fit spectra from the \pRT\added{ (pRT)} and \texttt{ATMO} retrievals fitting only the JWST/NIRSpec/G395H spectrum. Bottom: Best-fit spectra from \pRT  fitting the entire 0.2-5.1 $\mu$m HST+JWST spectrum. Black points show the spectrum with the offsets fitted in the free chemistry retrieval, while grey points show the spectrum with the offsets corresponding to the chemical equilibrium retrieval. While the self-consistent \added{PHOENIX radiative-convective equilibrium }model shows the correct features, it underpredicts the NUV absorption and overpredicts magnitude of the IR features. The flexibility of the retrievals allows them to fit the data well (i.e., $\chi^2/N < 1.2$). (*fig:spec_fits*)

</div>
<div id="div_fig2">

<img src="tmp_2503.15472/./W178b_pipeline_compare_v3_annotate.png" alt="Fig5" width="100%"/>

**Figure 5. -** Top: Transmission spectrum of WASP-178b from the \texttt{FIREFLy} and \texttt{transitspectroscopy} reductions. Middle: Transit depth difference between the \texttt{FIREFLy} and \texttt{transitspectroscopy} reductions, in units of the transit depth uncertainty. Solid lines show a second order polynomial fit to NRS1 and NRS2. Bottom: The \texttt{FIREFLy} reduction compared to two versions of the transit spectrum of ultra-hot Jupiter WASP-121b from \cite{gapp:2025}, after normalizing to each planet's equilibrium scale height and taking a 15-point moving average. The points with errorbars indicate the average measurement uncertainty. (*fig:spec_compare*)

</div>
<div id="div_fig3">

<img src="tmp_2503.15472/./W178b_abundance_profiles_v7.png" alt="Fig7" width="100%"/>

**Figure 7. -** Abundance profiles for $H_2$O, CO, SiO, and $CO_2$(clock-wise from upper-left) from the various retrieval scenarios. The abundance profile for WASP-121b from the \texttt{ATMO} chemical equilibrium retrieval is shown for comparison. Also shown are the normalized contribution functions (black) from the \texttt{pRT} chemical equilibrium retrieval, showing approximately where in the atmosphere each molecule is being measured. For SiO, the solid black contribution function shows where the 4.1 $\mu$m feature probes, while the dashed line shows where the NUV feature probes. The profile for the \texttt{ATMO} chemical equilibrium retrieval of SiO (dashed yellow) represents the 1-$\sigma$ upper-limit. (*fig:vmrs*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.15472"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Deep Mantle-Atmosphere Coupling and Carbonaceous Bombardment: Options for Biomolecule Formation on an Oxidized Early Earth

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.15479-b31b1b.svg)](https://arxiv.org/abs/2503.15479)<mark>Appeared on: 2025-03-20</mark> -  _Accepted for publication in The Astrophysical Journal. 44 pages, 12 figures (all colored)_

</div>
<div id="authors">

<mark>K. Paschek</mark>, et al. -- incl., <mark>T. K. Henning</mark>

</div>
<div id="abstract">

**Abstract:** $\noindent$ Understanding what environmental conditions prevailed on early Earth during the Hadean eon, and how this set the stage for the origins of life, remains a challenge. Geologic processes such as serpentinization and bombardment by chondritic material during the late veneer might have been very active, shaping an atmospheric composition reducing enough to allow efficient photochemical synthesis of $\ce{HCN}$ , one of the key precursors of prebiotic molecules. $\ce{HCN}$ can rain out and accumulate in warm little ponds (WLPs), forming prebiotic molecules such as nucleobases and the sugar ribose. These molecules could condense to nucleotides, the building blocks of RNA molecules, one of the ingredients of life. Here, we perform a systematic study of potential sources of reducing gases on Hadean Earth and calculate the concentrations of prebiotic molecules in WLPs based on a comprehensive geophysical and atmospheric model. We find that in a reduced $\ce{H2}$ -dominated atmosphere, carbonaceous bombardment can produce enough $\ce{HCN}$ to reach maximum WLP concentrations of $\sim$ $\SIrange{1}{10}{\milli\Molar}$ for nucleobases and, in the absence of seepage, $\sim$ $\SIrange{10}{100}{\micro\Molar}$ for ribose. If the Hadean atmosphere was initially oxidized and $\ce{CO2}$ -rich ( $\SI{90}{\percent}$ ), we find serpentinization alone can reduce the atmosphere, resulting in WLP concentrations of an order of magnitude lower than the reducing carbonaceous bombardment case. In both cases, concentrations are sufficient for nucleotide synthesis, as shown in experimental studies. RNA could have appeared on Earth immediately after it became habitable (about $\SI{100}{\mega\year}$ after the Moon-forming impact), or it could have (re)appeared later at any time up to the beginning of the Archean.

</div>

<div id="div_fig1">

<img src="tmp_2503.15479/./photochem_A1aB1aA1bB1bA1cB1c.png" alt="Fig6" width="100%"/>

**Figure 6. -** Effect of varying the hydrothermal circulation depth (HCD) during serpentinization on the atmospheric evolution. Shown is the simulated atmospheric composition of key species in the layer closest to the surface as a function of time. Only geological surface fluxes from extended models by Miyazaki2022 are considered as sources for \ce{H2}, \ce{CO2}, and \ce{CH4}(see cases MH/EH\_G0.5, MH/EH\_G1, and MH/EH\_G2 in Tables \ref{tab:cases_red} and \ref{tab:cases_ox}). The two epochs of the mid-Hadean (MH) at \SI{4.4}{\giga\year} with reducing initial conditions and the end-Hadean (EH) at \SI{4.0}{\giga\year} with oxidizing initial conditions are compared (left vs. right column; Panels **MH\_G0.5**, **MH\_G1**, **MH\_G2** vs. **EH\_G0.5**, **EH\_G1**, **EH\_G2**). For each epoch, different HCDs of the primordial ocean penetrating the crustal rock are considered (top row (**MH\_G0.5**, **EH\_G0.5**): \SI{0.5}{\kilo\meter}, middle row (**MH\_G1**, **EH\_G1**): \SI{1.0}{\kilo\meter}, bottom row (**MH\_G2**, **EH\_G2**): \SI{2.0}{\kilo\meter}). The initial conditions for the reducing and oxidizing models are summarized in Table \ref{tab:epochs}, closely following the established atmosphere models developed by Pearce2022. (*fig:photochem_geo*)

</div>
<div id="div_fig2">

<img src="tmp_2503.15479/./hadean_earth_scheme_new_revision.png" alt="Fig5" width="100%"/>

**Figure 5. -** The "**_\ce{HCN** machine_}": Geological, atmospheric, meteoritic, and chemical processes synthesizing the building blocks of life on the Hadean Earth (artist's impression, own creation, © Klaus Paschek). Panel **A**(lower right): Serpentinization and mantle processes lead to the efficient synthesis of \ce{H2} and \ce{CH4}, including reactions with water and \ce{CO2}. Panel **B**(lower left): Emission of \ce{H2}, \ce{CH4}, and \ce{CO2} from hydrothermal vents at volcanically active mid-ocean ridges. Panel **C**(center left): Degassing of \ce{H2} and \ce{HCN} by giant impacts. Panel **D**(top right): Synthesis of \ce{HCN} from \ce{CH4} and \ce{N2} by UV photochemistry in the atmosphere. Panel **E**(top left): Atmospheric \ce{HCN} rains out to the Earth's surface and enters lakes, ponds, and the ocean. In ponds, wet-dry cycling and aqueous chemistry convert \ce{HCN} into nucleobases, sugars, and ultimately RNA (oligo)nucleotides, key ingredients of life. (*fig:hadean_earth_scheme*)

</div>
<div id="div_fig3">

<img src="tmp_2503.15479/./photochem_A1cA2A3A4A5A6.png" alt="Fig7" width="100%"/>

**Figure 7. -** Testing whether exogenous chondritic bombardment is capable of keeping the atmosphere reduced while counteracting the oxidizing gases emitted by volcanism, as well as exploring its interplay with serpentinization. Shown is the simulated atmospheric composition of key species in the layer closest to the surface as a function of time starting at \SI{4.4}{\giga\year}. The atmosphere is initially set to a reducing state (\ce{H2}: \SI{90}{\percent}, \ce{N2}: \SI{10}{\percent}, \ce{CH4}: \SI{1}{ppm}) and a temperature of \SI{78}{\celsius}(Table \ref{tab:epochs}), closely following the reducing case considered by Pearce2022. Panel/case **MH\_G2** corresponds to a model driven by serpentinization alone (geology only) and is identical to Figure \ref{fig:photochem_geo}(MH\_G2). It is shown here again for reference to facilitate comparison with the other cases. Cases E and C correspond to scenarios without any contribution from serpentinization, but with the reduction capacity of the late veneer alone competing with the \ce{CO2} flux emitted from the Earth's mantle. Panel **MH\_E** corresponds to a scenario with a bombardment of pure enstatite composition, and panel **MH\_C** instead corresponds to a pure carbonaceous composition. Panels **MH\_G2E**, **MH\_G2C**, and **MH\_G2EC** present the results corresponding to scenarios combining geological and late veneer source flux contributions, with case MH\_G2E combining the geology with an enstatite bombardment (cases MH\_G2 and MH\_E), case MH\_G2C combines the geology instead with a carbonaceous bombardment (MH\_G2 and MH\_C), and case MH\_G2EC combines all three with a half-half split bombardment composition (MH\_G2, \SI{50}{\percent} of MH\_E, and \SI{50}{\percent} of MH\_C). (*fig:photochem_red*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.15479"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

393  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

32  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

26  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
